In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3 

# read data
data = pd.read_excel(r'c:\Users\HP\Documents\edu\data mining\end  sem exams\online+retail\Online Retail.xlsx')
data.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [3]:

def data_cleaning(data):
    print('Data Cleaning Process Initiated...')
    # check for missing vals
    missing= data.isnull().sum()
    print(f'No of missing : {missing}')
    #check for duplicate rows
    dups= data.duplicated().sum()
    print(f'No of duplicate rows: {dups}')
    # missing vals change
    print(f'the no. of missing dealtith:  ') 
    # Work on missing values
    data['CustomerID'].fillna(0, inplace= True) # will help in trend analysis, remember to remove for any customer analysis
    data.dropna(subset=['Description'], inplace=True)# filling won't help much here as description is important
    # work on duplicate rows only delete when entire row is duplicate
    data.drop_duplicates(keep='first', inplace=True)
    # confimations
    missing_n= data.isnull().sum()
    print(f'No of missing : {missing_n}')
    #check for duplicate rows
    dups_n= data.duplicated().sum()
    print(f'No of duplicate rows: {dups_n}')
    # missing vals change
    print(f'the no. of missing dealtith: {missing}-{missing_n}', '\n') 
    # Duplicates dealtwith
    print(f'the no. of duplicates dealtith: {dups}-{dups_n}', '\n')
    # total number of rows
    total_rows= data.shape[0]
    print(f'Total no of rows: {total_rows}')

    return data


In [4]:
def data_transformations(data):
    print('Data Transformation Process Initiated...')
    # transformations
    # add total price column
    data['TotalPrice']= data['Quantity'] * data['UnitPrice']
    # convert InvoiceDate to datetime
    data['InvoiceDate']= pd.to_datetime(data['InvoiceDate'])
    # have only previous year data since data has onlu 2011 and 2010 data well yse current year as 2012
    start_date = pd.to_datetime('2011-01-01') 
    end_date = pd.to_datetime('2012-01-01')
    data = data[(data['InvoiceDate'] >= start_date) & (data['InvoiceDate'] < end_date)]
    # count of outliers
    outlier_qty= data[data['Quantity'] <=0].shape[0]
    outlier_price= data[data['UnitPrice'] <=0].shape[0]
    # handle outliers
    data= data[(data['Quantity'] >0) & (data['UnitPrice'] >0)]
    # outliers removed
    print(f'No of outlier rows removed - Quantity: {outlier_qty}, UnitPrice: {outlier_price}', '\n')
    # reser index
    data.reset_index(drop=True, inplace=True)
    return data


In [5]:
def data_csv(data):
    # rewrite data to csv to prevent long runtime of excel read and write data to sql db
    data.to_csv(r'c:\Users\HP\Documents\edu\data mining\end  sem exams\online+retail\Online Retail.csv', index=False)
    # read data
    data= pd.read_csv(r'c:\Users\HP\Documents\edu\data mining\end  sem exams\online+retail\Online Retail.csv')
    return data


def data_to_sql():
    # make db file
    OnlineRetail_db = r'c:\Users\HP\Documents\edu\data mining\end  sem exams\online+retail\OnlineRetail.db'
    return  OnlineRetail_db


In [6]:
# connection function
def create_connection(onlineRetail_db):
    conn= sqlite3.connect(r'c:\Users\HP\Documents\edu\data mining\end  sem exams\online+retail\OnlineRetail.db')
    if conn is not None:
        print("Connection to database is successful")
    else :
        print("Connection to database failed")    
    return conn


In [7]:
def querries(conn):
    # make fact table using sqlite3(there is only one table so no joins needed)
    fact_table_query="""
    SELECT InvoiceNo,CustomerID,StockCode,TotalPrice,InvoiceDate FROM OnlineRetail
    """
    #  customer dimension table
    customer_info_query= """
    SELECT DISTINCT CustomerID, Country
    FROM OnlineRetail
    WHERE CustomerID != 0
    """
    # product dimension table
    product_info__query="""
    SELECT DISTINCT StockCode, Description, UnitPrice
    FROM OnlineRetail
    """

    Sales_fact= pd.read_sql_query(fact_table_query, conn)
    print('Sales fact table',Sales_fact.head())
    Customer_dim= pd.read_sql_query(customer_info_query, conn)
    print('Customer data table',Customer_dim.head())
    Product_dim= pd.read_sql_query(product_info__query, conn)
    print('Product data table', Product_dim.head())
    return Sales_fact, Customer_dim, Product_dim



 

In [8]:
# combined etl_process
def etl_process(data):
    data = data_cleaning(data)
    data = data_transformations(data)
    data = data_csv(data)
    onlineRetail_db = data_to_sql()
    conn = create_connection(onlineRetail_db)
    # write dataframe to SQL (data is a DataFrame here)
    data.to_sql('OnlineRetail', conn, if_exists='replace', index=False)
    print('tables created successfully')
    Sales_fact, Customer_dim, Product_dim = querries(conn)
    conn.close()
    return Sales_fact, Customer_dim, Product_dim
Sales_fact, Customer_dim, Product_dim = etl_process(data)

# save the tables to csv files
Sales_fact.to_csv(r'c:\Users\HP\Documents\edu\data mining\end  sem exams\online+retail\Sales_fact.csv', index=False)
Customer_dim.to_csv(r'c:\Users\HP\Documents\edu\data mining\end  sem exams\online+retail\Customer_dim.csv', index=False)
Product_dim.to_csv(r'c:\Users\HP\Documents\edu\data mining\end  sem exams\online+retail\Product_dim.csv', index=False)

Data Cleaning Process Initiated...
No of missing : InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
No of duplicate rows: 5268
the no. of missing dealtith:  


C:\Users\HP\AppData\Local\Temp\ipykernel_8780\4207034945.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['CustomerID'].fillna(0, inplace= True) # will help in trend analysis, remember to remove for any customer analysis


No of missing : InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64
No of duplicate rows: 0
the no. of missing dealtith: InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64-InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64 

the no. of duplicates dealtith: 5268-0 

Total no of rows: 535187
Data Transformation Process Initiated...
No of outlier rows removed - Quantity: 8998, UnitPrice: 912 



C:\Users\HP\AppData\Local\Temp\ipykernel_8780\3167239171.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv(r'c:\Users\HP\Documents\edu\data mining\end  sem exams\online+retail\Online Retail.csv')


Connection to database is successful
tables created successfully
Sales fact table   InvoiceNo  CustomerID StockCode  TotalPrice          InvoiceDate
0    539993     13313.0     22386        19.5  2011-01-04 10:00:00
1    539993     13313.0     21499        10.5  2011-01-04 10:00:00
2    539993     13313.0     21498        10.5  2011-01-04 10:00:00
3    539993     13313.0     22379        10.5  2011-01-04 10:00:00
4    539993     13313.0     20718        12.5  2011-01-04 10:00:00
Customer data table    CustomerID         Country
0     13313.0  United Kingdom
1     18097.0  United Kingdom
2     16656.0  United Kingdom
3     16875.0  United Kingdom
4     13094.0  United Kingdom
Product data table   StockCode                Description  UnitPrice
0     22386    JUMBO BAG PINK POLKADOT       1.95
1     21499         BLUE POLKADOT WRAP       0.42
2     21498        RED RETROSPOT WRAP        0.42
3     22379   RECYCLING BAG RETROSPOT        2.10
4     20718  RED RETROSPOT SHOPPER BAG       1.